In [29]:
#Handling imbalanced datasets in machine learning

from __future__ import division, print_function, unicode_literals
import numpy as np
import pandas as pd
from pandas import read_csv
import math
import sklearn
from sklearn import linear_model
import random
from sklearn import cluster
from sklearn.naive_bayes import GaussianNB
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import matplotlib
import matplotlib.pyplot as plt

In [30]:
#loading data

creditData=np.loadtxt('creditcard.csv',dtype=np.str,delimiter=',',skiprows=1)

In [31]:
print(creditData.shape)

#stripping "" from output variable
creditData[:,creditData.shape[1]-1]=np.core.defchararray.strip(creditData[:,creditData.shape[1]-1], chars='"')
#type conversion
creditData=creditData.astype(np.float)
print(creditData.dtype)

(284807, 31)
float64


In [32]:
#1. Data Level Approach
fraudRate=(np.count_nonzero(creditData[:,creditData.shape[1]-1])/creditData.shape[0])
print(fraudRate)#ratio of non fraud to fraud data

0.001727485630620034


In [74]:
#splitting between train and test set
xTrain, xTest, yTrain, yTest=sklearn.model_selection.train_test_split(creditData[:,:creditData.shape[1]-1],creditData[:,creditData.shape[1]-1],test_size=0.20)

#test contains test input and output both
test=pd.DataFrame(xTest)
test[test.shape[1]]=yTest
testF=test[test[test.shape[1]-1]==1]#test data for fraud examples
testNf=test[test[test.shape[1]-1]==0]#test data for non fraud examples


train=pd.DataFrame(xTrain)
train[train.shape[1]]=yTrain
trainF=train[train[train.shape[1]-1]==1]#train data for fraud examples
trainNf=train[train[train.shape[1]-1]==0]#train data for non fraud examples

#reducing the size of data
train=pd.DataFrame()
xTrain=pd.DataFrame()
yTrain=pd.DataFrame()
for i in range(30000):
    r=np.random.randint(0,trainNf.shape[0])
    train=train.append(trainNf.iloc[r,:])  
for i in range(3000):
    r=np.random.randint(0,trainF.shape[0])
    train=train.append(trainF.iloc[r,:])

xTrain=xTrain.append(train.loc[:,:train.shape[1]-2])
yTrain[train.shape[1]-1]=train.loc[:,train.shape[1]-1]

trainF=train[train[train.shape[1]-1]==1]
trainNf=train[train[train.shape[1]-1]==0]

testN=pd.DataFrame()
for i in range(testF.shape[0]):
    r=np.random.randint(0,testNf.shape[0])
    testN=testN.append(testNf.iloc[r,:])
testNf=testN
print(xTrain.shape,yTrain.shape,trainF.shape,trainNf.shape,testF.shape,testNf.shape)

'''#tsne
Xnew=TSNE(n_components=2).fit_transform(xTrain)
#print("K means for t-SNE")

#kmeans = KMeans(n_clusters=2).fit(Xnew)
#y=kmeans.predict(Xnew)
fig = plt.figure(figsize=(10,10))
#matplotlib.pyplot.scatter(Xnew[:,0], Xnew[:,1], c=y, cmap='tab10')
colors = ['blue','red']
matplotlib.pyplot.scatter(Xnew[:,0], Xnew[:,1], c=np.ravel(yTrain.astype(np.int)), cmap=matplotlib.colors.ListedColormap(colors))
#plt.title('Clustering of points after running the kmeans algorithm on tSNE redued data')
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()'''

(33000, 30) (33000, 1) (3000, 31) (30000, 31) (96, 31) (96, 31)


'#tsne\nXnew=TSNE(n_components=2).fit_transform(xTrain)\n#print("K means for t-SNE")\n\n#kmeans = KMeans(n_clusters=2).fit(Xnew)\n#y=kmeans.predict(Xnew)\nfig = plt.figure(figsize=(10,10))\n#matplotlib.pyplot.scatter(Xnew[:,0], Xnew[:,1], c=y, cmap=\'tab10\')\ncolors = [\'blue\',\'red\']\nmatplotlib.pyplot.scatter(Xnew[:,0], Xnew[:,1], c=np.ravel(yTrain.astype(np.int)), cmap=matplotlib.colors.ListedColormap(colors))\n#plt.title(\'Clustering of points after running the kmeans algorithm on tSNE redued data\')\nplt.xlabel(\'x1\')\nplt.ylabel(\'x2\')\nplt.show()'

In [75]:
#Random undersampling
train=pd.DataFrame()
for i in range(np.count_nonzero(yTrain)):
    r=np.random.randint(0,trainNf.shape[0])
    train=train.append(trainNf.iloc[r,:])  
train=train.append(trainF)


In [35]:
#prototype based classification on original data(bad classification with eucledian distance metric)

clf=sklearn.neighbors.NearestCentroid().fit(xTrain,np.ravel(yTrain))

print(clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples

#prototype based classification on random undersampled data
clf=sklearn.neighbors.NearestCentroid().fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print(clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples

0.43564356435643564
0.6435643564356436
0.43564356435643564
0.6435643564356436


In [36]:
#prototype based classification on original data(classification with minkowski distance with p=-1)

def dist(x,y):
    x=abs(x-y)
    x=x**(-0.1)
    s=np.sum(x)
    s=s**(-1/0.1)
    return s
clf=sklearn.neighbors.NearestCentroid(metric=dist).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#prototype based classification on random undersampled data
clf=sklearn.neighbors.NearestCentroid(metric=dist).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

accuracy on non fraud samples  1.0
recall  0.8118811881188119
precision  1.0
F1 score  0.8961748633879781
accuracy on non fraud samples  1.0
recall  0.7920792079207921
precision  1.0
F1 score  0.8839779005524863


/home/lenovo/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/nearest_centroid.py:140: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


In [37]:
#Naive bayes classifier

clf=GaussianNB().fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#naive bayes classification on random undersampled data
clf=GaussianNB().fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

accuracy on non fraud samples  1.0
recall  0.7326732673267327
precision  1.0
F1 score  0.8457142857142858
accuracy on non fraud samples  1.0
recall  0.7722772277227723
precision  1.0
F1 score  0.8715083798882681


In [38]:
#Decision tree classifier

clf = sklearn.tree.DecisionTreeClassifier().fit(xTrain,yTrain)

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


#decision tree classification on random undersampled data
clf=sklearn.tree.DecisionTreeClassifier().fit(train.loc[:,:train.shape[1]-2],train.loc[:,train.shape[1]-1])
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

clf = sklearn.tree.DecisionTreeClassifier(class_weight={1:10}).fit(xTrain,yTrain)

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

accuracy on non fraud samples  1.0
recall  0.8811881188118812
precision  1.0
F1 score  0.9368421052631579
accuracy on non fraud samples  0.9900990099009901
recall  0.8811881188118812
precision  0.9888888888888889
F1 score  0.9319371727748692
accuracy on non fraud samples  1.0
recall  0.8415841584158416
precision  1.0
F1 score  0.913978494623656


In [76]:
# K- nearest neighbor

clf = sklearn.neighbors.KNeighborsClassifier(metric=dist).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#K nearest neighbor classification on random undersampled data
clf=sklearn.neighbors.KNeighborsClassifier(metric=dist).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precision
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

/home/lenovo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in power
  


accuracy on non fraud samples  0.9895833333333334
recall  0.3125
precision  0.967741935483871
F1 score  0.47244094488188976
accuracy on non fraud samples  0.90625
recall  0.7708333333333334
precision  0.891566265060241
F1 score  0.8268156424581005


In [40]:
#logistic regression on original data
#class sklearn.linear_model.LogisticRegression(penalty=’l2’, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver=’warn’, max_iter=100, multi_class=’warn’, verbose=0, warm_start=False, n_jobs=None)
clf=linear_model.LogisticRegression().fit(xTrain,np.ravel(yTrain))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#logistic regression on random undersampled data
clf=linear_model.LogisticRegression().fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print(TN,TP,FP)
print("precision ",precision)#precision
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

clf=linear_model.LogisticRegression(class_weight={1:10}).fit(xTrain,np.ravel(yTrain))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

accuracy on non fraud samples  0.9900990099009901
recall  0.8712871287128713
precision  0.9887640449438202
F1 score  0.9263157894736842
accuracy on non fraud samples  0.9702970297029703
recall  0.9405940594059405
98.0 95.0 3.0
precision  0.9693877551020408
F1 score  0.9547738693467337
accuracy on non fraud samples  0.9900990099009901
recall  0.9405940594059405
precision  0.9895833333333334
F1 score  0.964467005076142


In [41]:
#SVM on original data
clf=sklearn.svm.SVC(kernel='linear',cache_size=4096,max_iter=200,verbose=True).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#SVM on random undersampled data
clf=sklearn.svm.SVC(kernel='linear',cache_size=4096,max_iter=200,verbose=True).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

clf=sklearn.svm.SVC(kernel='linear',class_weight={1:10},cache_size=4096,verbose=True).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


[LibSVM]accuracy on non fraud samples  0.5544554455445545
recall  0.36633663366336633
precision  0.45121951219512196
F1 score  0.40437158469945356
[LibSVM]accuracy on non fraud samples  0.43564356435643564
recall  0.6336633663366337
precision  0.5289256198347108
F1 score  0.5765765765765766
[LibSVM]

/home/lenovo/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


accuracy on non fraud samples  1.0
recall  0.9207920792079208
precision  1.0
F1 score  0.9587628865979382


In [77]:
#Random Oversampling

train=pd.DataFrame()
for i in range(yTrain.shape[0]-2*np.count_nonzero(yTrain)):
    r=np.random.randint(0,trainF.shape[0])
    train=train.append(trainF.iloc[r,:])  
train=train.append(trainNf)

In [43]:
#prototype based classification on original data(classification with minkowski distance)


def dist(x,y):
    x=abs(x-y)
    x=x**(-0.1)
    s=np.sum(x)
    s=s**(-1/0.1)
    return s
clf=sklearn.neighbors.NearestCentroid(metric=dist).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#prototype based classification on random undersampled data
clf=sklearn.neighbors.NearestCentroid(metric=dist).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

accuracy on non fraud samples  1.0
recall  0.8118811881188119
precision  1.0
F1 score  0.8961748633879781
accuracy on non fraud samples  1.0
recall  0.8118811881188119
precision  1.0
F1 score  0.8961748633879781


/home/lenovo/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/nearest_centroid.py:140: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


In [44]:
#Naive bayes classifier

clf=GaussianNB().fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#naive bayes classification on random undersampled data
clf=GaussianNB().fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

accuracy on non fraud samples  1.0
recall  0.7326732673267327
precision  1.0
F1 score  0.8457142857142858
accuracy on non fraud samples  1.0
recall  0.7524752475247525
precision  1.0
F1 score  0.8587570621468926


In [45]:
#Decision tree classifier

clf = sklearn.tree.DecisionTreeClassifier().fit(xTrain,yTrain)

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#decision tree classification on random undersampled data
clf=sklearn.tree.DecisionTreeClassifier().fit(train.loc[:,:train.shape[1]-2],train.loc[:,train.shape[1]-1])
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

accuracy on non fraud samples  1.0
recall  0.8712871287128713
precision  1.0
F1 score  0.9312169312169313
accuracy on non fraud samples  1.0
recall  0.8613861386138614
precision  1.0
F1 score  0.9255319148936171


In [78]:
# K- nearest neighbor

clf = sklearn.neighbors.KNeighborsClassifier(metric=dist).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#K nearest neighbor classification on random undersampled data
clf=sklearn.neighbors.KNeighborsClassifier(metric=dist).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

/home/lenovo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in power
  


accuracy on non fraud samples  0.9895833333333334
recall  0.3125
precision  0.967741935483871
F1 score  0.47244094488188976
accuracy on non fraud samples  0.9895833333333334
recall  0.3541666666666667
precision  0.9714285714285714
F1 score  0.5190839694656488


In [47]:
#logistic regression on original data
#class sklearn.linear_model.LogisticRegression(penalty=’l2’, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver=’warn’, max_iter=100, multi_class=’warn’, verbose=0, warm_start=False, n_jobs=None)
clf=linear_model.LogisticRegression().fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#logistic regression on random undersampled data
clf=linear_model.LogisticRegression().fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

accuracy on non fraud samples  0.9900990099009901
recall  0.8712871287128713
precision  0.9887640449438202
F1 score  0.9263157894736842
accuracy on non fraud samples  0.9801980198019802
recall  0.9405940594059405
precision  0.979381443298969
F1 score  0.9595959595959594


In [48]:
#SVM on original data
clf=sklearn.svm.SVC(kernel='linear',cache_size=4096,max_iter=200,verbose=True).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))
               
#SVM on random undersampled data
clf=sklearn.svm.SVC(kernel='linear',cache_size=4096,max_iter=200,verbose=True).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

[LibSVM]accuracy on non fraud samples  0.5544554455445545
recall  0.36633663366336633
precision  0.45121951219512196
F1 score  0.40437158469945356
[LibSVM]accuracy on non fraud samples  0.5544554455445545
recall  0.36633663366336633
precision  0.45121951219512196
F1 score  0.40437158469945356


/home/lenovo/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [79]:
#k means on non fraud data with number of centres=number of fraud data
clf=cluster.KMeans(n_clusters=trainF.shape[0]).fit(trainNf.loc[:,:trainNf.shape[1]-2])
train=pd.DataFrame(clf.cluster_centers_)
train[train.shape[1]]=np.zeros(train.shape[0])
train=train.append(trainF)
print(train.shape)

(6000, 31)


In [50]:
#prototype based classification on original data(classification with minkowski distance)


def dist(x,y):
    x=abs(x-y)
    x=x**(-0.1)
    s=np.sum(x)
    s=s**(-1/0.1)
    return s
clf=sklearn.neighbors.NearestCentroid(metric=dist).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#prototype based classification on random undersampled data
clf=sklearn.neighbors.NearestCentroid(metric=dist).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

accuracy on non fraud samples  1.0
recall  0.8118811881188119
precision  1.0
F1 score  0.8961748633879781
accuracy on non fraud samples  1.0
recall  0.8217821782178217
precision  1.0
F1 score  0.9021739130434783


/home/lenovo/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/nearest_centroid.py:140: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


In [51]:
#Naive bayes classifier

clf=GaussianNB().fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#naive bayes classification on random undersampled data
clf=GaussianNB().fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


accuracy on non fraud samples  1.0
recall  0.7326732673267327
precision  1.0
F1 score  0.8457142857142858
accuracy on non fraud samples  1.0
recall  0.8118811881188119
precision  1.0
F1 score  0.8961748633879781


In [52]:
#Decision tree classifier

clf = sklearn.tree.DecisionTreeClassifier().fit(xTrain,yTrain)

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#decision tree classification on random undersampled data
clf=sklearn.tree.DecisionTreeClassifier().fit(train.loc[:,:train.shape[1]-2],train.loc[:,train.shape[1]-1])
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


accuracy on non fraud samples  1.0
recall  0.8811881188118812
precision  1.0
F1 score  0.9368421052631579
accuracy on non fraud samples  0.7920792079207921
recall  0.9207920792079208
precision  0.8157894736842105
F1 score  0.8651162790697674


In [80]:
# K- nearest neighbor

clf = sklearn.neighbors.KNeighborsClassifier(metric=dist).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#K nearest neighbor classification on random undersampled data
clf=sklearn.neighbors.KNeighborsClassifier(metric=dist).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

/home/lenovo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in power
  


accuracy on non fraud samples  0.9895833333333334
recall  0.3125
precision  0.967741935483871
F1 score  0.47244094488188976
accuracy on non fraud samples  0.8541666666666666
recall  0.9166666666666666
precision  0.8627450980392157
F1 score  0.888888888888889


In [54]:
#logistic regression on original data
#class sklearn.linear_model.LogisticRegression(penalty=’l2’, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver=’warn’, max_iter=100, multi_class=’warn’, verbose=0, warm_start=False, n_jobs=None)
clf=linear_model.LogisticRegression().fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#logistic regression on random undersampled data
clf=linear_model.LogisticRegression().fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


accuracy on non fraud samples  0.9900990099009901
recall  0.8712871287128713
precision  0.9887640449438202
F1 score  0.9263157894736842
accuracy on non fraud samples  0.9504950495049505
recall  0.9306930693069307
precision  0.9494949494949495
F1 score  0.9400000000000001


In [55]:
#SVM on original data
clf=sklearn.svm.SVC(kernel='linear',cache_size=4096,max_iter=200,verbose=True).fit(xTrain,np.ravel(yTrain))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#SVM on random undersampled data
clf=sklearn.svm.SVC(kernel='linear',cache_size=4096,max_iter=200,verbose=True).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


[LibSVM]accuracy on non fraud samples  0.5544554455445545
recall  0.36633663366336633
precision  0.45121951219512196
F1 score  0.40437158469945356
[LibSVM]accuracy on non fraud samples  0.6336633663366337
recall  0.26732673267326734
precision  0.421875
F1 score  0.32727272727272727


/home/lenovo/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [81]:
#cluster based oversampling
dataCluster=pd.DataFrame()
dataCluster=dataCluster.append(trainNf)
#making clusters
clf1=cluster.KMeans(n_clusters=20).fit(trainNf.loc[:,:trainNf.shape[1]-2])
clusterTotal=pd.DataFrame(clf1.predict(trainNf.loc[:,:trainNf.shape[1]-2]))
dataCluster[31]=np.array(clusterTotal)
clusterTotal1=clusterTotal.groupby([0]).size()
clusterMax1=clusterTotal1.max()

clf2=cluster.KMeans(n_clusters=2).fit(trainF.loc[:,:trainF.shape[1]-2])
clusterTotal=pd.DataFrame(clf2.predict(trainF.loc[:,:trainF.shape[1]-2]))
clusterTotal2=clusterTotal.groupby([0]).size()
clusterMax2=20*clusterMax1//2

#oversampling data of each cluster

dataCluster=dataCluster.sort_values(by=trainNf.shape[1]-1)
train=pd.DataFrame()
train=train.append(trainNf)
currind=0
for i in range(20):
    for j in (clusterMax1-clusterTotal1):
        r=np.random.randint(0,clusterTotal1[i])
        train=train.append(dataCluster.iloc[r+currind,:31])
    currind=currind+clusterTotal1[i]
#print(train.head)

dataCluster=pd.DataFrame(trainF)
dataCluster=dataCluster.sort_values(by=trainF.shape[1]-1)
train=train.append(trainF)
currind=0
for i in range(2):
    for j in range(clusterMax2):
        r=np.random.randint(0,clusterTotal2[i])
        train=train.append(dataCluster.iloc[r+currind,:31])
    currind=currind+clusterTotal2[i]
print(train.groupby([train.shape[1]-1]).size())

30
0.0    30400
1.0    46640
dtype: int64


In [57]:
#prototype based classification on original data(classification with minkowski distance)


def dist(x,y):
    x=abs(x-y)
    x=x**(-0.1)
    s=np.sum(x)
    s=s**(-1/0.1)
    return s
clf=sklearn.neighbors.NearestCentroid(metric=dist).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#prototype based classification on random undersampled data
clf=sklearn.neighbors.NearestCentroid(metric=dist).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


accuracy on non fraud samples  1.0
recall  0.8118811881188119
precision  1.0
F1 score  0.8961748633879781
accuracy on non fraud samples  1.0
recall  0.8118811881188119
precision  1.0
F1 score  0.8961748633879781


/home/lenovo/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/nearest_centroid.py:140: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


In [58]:
#Naive bayes classifier

clf=GaussianNB().fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#naive bayes classification on random undersampled data
clf=GaussianNB().fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


accuracy on non fraud samples  1.0
recall  0.7326732673267327
precision  1.0
F1 score  0.8457142857142858
accuracy on non fraud samples  1.0
recall  0.7821782178217822
precision  1.0
F1 score  0.8777777777777778


In [59]:
#Decision tree classifier

clf = sklearn.tree.DecisionTreeClassifier().fit(xTrain,yTrain)

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#decision tree classification on random undersampled data
clf=sklearn.tree.DecisionTreeClassifier().fit(train.loc[:,:train.shape[1]-2],train.loc[:,train.shape[1]-1])
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


accuracy on non fraud samples  1.0
recall  0.8811881188118812
precision  1.0
F1 score  0.9368421052631579
accuracy on non fraud samples  1.0
recall  0.8415841584158416
precision  1.0
F1 score  0.913978494623656


In [82]:
# K- nearest neighbor

clf = sklearn.neighbors.KNeighborsClassifier(metric=dist).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


#K nearest neighbor classification on random undersampled data
clf=sklearn.neighbors.KNeighborsClassifier(metric=dist).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

/home/lenovo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in power
  


accuracy on non fraud samples  0.9895833333333334
recall  0.3125
precision  0.967741935483871
F1 score  0.47244094488188976
accuracy on non fraud samples  0.9895833333333334
recall  0.53125
precision  0.9807692307692307
F1 score  0.6891891891891891


In [61]:
#logistic regression on original data
#class sklearn.linear_model.LogisticRegression(penalty=’l2’, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver=’warn’, max_iter=100, multi_class=’warn’, verbose=0, warm_start=False, n_jobs=None) 
clf=linear_model.LogisticRegression().fit(xTrain,np.ravel(yTrain))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#logistic regression on random undersampled data 
clf=linear_model.LogisticRegression().fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]) )
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


accuracy on non fraud samples  0.9900990099009901
recall  0.8712871287128713
precision  0.9887640449438202
F1 score  0.9263157894736842
accuracy on non fraud samples  0.9801980198019802
recall  0.9405940594059405
precision  0.979381443298969
F1 score  0.9595959595959594


In [62]:
#SVM

#SVM on original data
clf=sklearn.svm.SVC(kernel='linear',cache_size=4096,max_iter=200,verbose=True).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#SVM on random undersampled data
clf=sklearn.svm.SVC(kernel='linear',cache_size=4096,max_iter=200,verbose=True).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


[LibSVM]accuracy on non fraud samples  0.5544554455445545
recall  0.36633663366336633
precision  0.45121951219512196
F1 score  0.40437158469945356
[LibSVM]accuracy on non fraud samples  0.45544554455445546
recall  0.6336633663366337
precision  0.5378151260504201
F1 score  0.5818181818181819


/home/lenovo/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [83]:
#smote
train=pd.DataFrame(trainF)
print(train.shape)
#print(trainF.iloc[0,:trainF.shape[1]-2])
for i in range(yTrain.shape[0]-2*np.count_nonzero(yTrain)):
    j=np.random.randint(0,trainF.shape[0])
    neigh=sklearn.neighbors.KNeighborsClassifier(n_neighbors=1).fit(trainF.loc[:,:trainF.shape[1]-2],trainF.loc[:,trainF.shape[1]-1])
    neighbour=neigh.kneighbors(np.reshape(np.array(trainF.iloc[j,:trainF.shape[1]-1]),(1,-1)), 2, False)
    p=np.random.random()
    newPoint=(trainF.iloc[j,:trainF.shape[1]-1])+p*(trainF.iloc[neighbour[0][1],:trainF.shape[1]-1]-trainF.iloc[j,:trainF.shape[1]-1])
    newPoint=newPoint.append(pd.Series(1))
    newPoint=newPoint.reset_index(drop=True)
    train=train.append(newPoint,ignore_index=True)
print(train.shape)
train=train.append(trainNf)
print(train.shape)

(3000, 31)
(30000, 31)
(60000, 31)


In [64]:
#prototype based classification on original data(classification with minkowski distance)


def dist(x,y):
    x=abs(x-y)
    x=x**(-0.1)
    s=np.sum(x)
    s=s**(-1/0.1)
    return s
clf=sklearn.neighbors.NearestCentroid(metric=dist).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#prototype based classification on random undersampled data
clf=sklearn.neighbors.NearestCentroid(metric=dist).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


accuracy on non fraud samples  1.0
recall  0.8118811881188119
precision  1.0
F1 score  0.8961748633879781
accuracy on non fraud samples  1.0
recall  0.8217821782178217
precision  1.0
F1 score  0.9021739130434783


/home/lenovo/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/nearest_centroid.py:140: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


In [65]:
#Naive bayes classifier

clf=GaussianNB().fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#naive bayes classification on random undersampled data
clf=GaussianNB().fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


accuracy on non fraud samples  1.0
recall  0.7326732673267327
precision  1.0
F1 score  0.8457142857142858
accuracy on non fraud samples  1.0
recall  0.7623762376237624
precision  1.0
F1 score  0.8651685393258427


In [66]:
#Decision tree classifier

clf = sklearn.tree.DecisionTreeClassifier().fit(xTrain,yTrain)

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#decision tree classification on random undersampled data
clf=sklearn.tree.DecisionTreeClassifier().fit(train.loc[:,:train.shape[1]-2],train.loc[:,train.shape[1]-1])
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


accuracy on non fraud samples  1.0
recall  0.8811881188118812
precision  1.0
F1 score  0.9368421052631579
accuracy on non fraud samples  1.0
recall  0.8415841584158416
precision  1.0
F1 score  0.913978494623656


In [84]:
# K- nearest neighbor

clf = sklearn.neighbors.KNeighborsClassifier(metric=dist).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#K nearest neighbor classification on random undersampled data
clf=sklearn.neighbors.KNeighborsClassifier(metric=dist).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

/home/lenovo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in power
  


accuracy on non fraud samples  0.9895833333333334
recall  0.3125
precision  0.967741935483871
F1 score  0.47244094488188976
accuracy on non fraud samples  0.9895833333333334
recall  0.53125
precision  0.9807692307692307
F1 score  0.6891891891891891


In [68]:
#logistic regression on original data
#class sklearn.linear_model.LogisticRegression(penalty=’l2’, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver=’warn’, max_iter=100, multi_class=’warn’, verbose=0, warm_start=False, n_jobs=None)
clf=linear_model.LogisticRegression().fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#logistic regression on random undersampled data
clf=linear_model.LogisticRegression().fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


accuracy on non fraud samples  0.9900990099009901
recall  0.8712871287128713
precision  0.9887640449438202
F1 score  0.9263157894736842
accuracy on non fraud samples  0.9702970297029703
recall  0.9405940594059405
precision  0.9693877551020408
F1 score  0.9547738693467337


In [69]:
#SVM on original data
clf=sklearn.svm.SVC(kernel='linear',cache_size=4096,max_iter=200,verbose=True).fit(xTrain,np.ravel(yTrain))

print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))

#SVM on random undersampled data
clf=sklearn.svm.SVC(kernel='linear',cache_size=4096,max_iter=200,verbose=True).fit(train.loc[:,:train.shape[1]-2],np.ravel(train.loc[:,train.shape[1]-1]))
print("accuracy on non fraud samples ",clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1]))#accuracy on non fraud examples
print("recall ", clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))#accuracy on fraud examples(recall)
TP=clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])*(testF.shape[0])
TN=clf.score(testNf.loc[:,:testNf.shape[1]-2],testNf.loc[:,testNf.shape[1]-1])*(testNf.shape[0])
FP=(testNf.shape[0])-TN
precision=(TP)/(TP+FP)
print("precision ",precision)#precisio
print("F1 score ",2*precision*clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1])/(precision+(clf.score(testF.loc[:,:testF.shape[1]-2],testF.loc[:,testF.shape[1]-1]))))


[LibSVM]accuracy on non fraud samples  0.5544554455445545
recall  0.36633663366336633
precision  0.45121951219512196
F1 score  0.40437158469945356
[LibSVM]accuracy on non fraud samples  0.5544554455445545
recall  0.36633663366336633
precision  0.45121951219512196
F1 score  0.40437158469945356


/home/lenovo/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
